In [1]:
from Metaheuristicas.fitness_functions import *


In [2]:
X, y = load_and_preprocess_data(filename='Resources/SeisBenchV1_v1_1.json')


In [3]:
from Metaheuristicas.Genetico import genetic_algorithm

mutation = 0.1
crossover = 0.9

In [4]:
import pandas as pd
from IPython.display import display, clear_output

# Step 1: Initialize empty DataFrames for each classifier with metrics as columns
metrics = ["Accuracy", "Precision", "Recall", "F1 Score", "AUC"]

naive_bayes_df = pd.DataFrame(columns=metrics, index=["Mutual Information", "X2", "Relief"])
random_forest_df = pd.DataFrame(columns=metrics, index=["Mutual Information", "X2", "Relief"])
neural_network_df = pd.DataFrame(columns=metrics, index=["Mutual Information", "X2", "Relief"])

# Display all tables function
def display_tables():
    clear_output(wait=True)
    print("Naive Bayes Results")
    display(naive_bayes_df)
    print("Random Forest Results")
    display(random_forest_df)
    print("Neural Network Results")
    display(neural_network_df)

In [5]:
def add_result(classifier, fitness_function, accuracy, precision, recall, f1_score, auc):
    new_data = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "AUC": auc
    }

    if classifier == "Naive Bayes":
        global naive_bayes_df
        naive_bayes_df.loc[fitness_function] = new_data
    elif classifier == "Random Forest":
        global random_forest_df
        random_forest_df.loc[fitness_function] = new_data
    elif classifier == "Neural Network":
        global neural_network_df
        neural_network_df.loc[fitness_function] = new_data

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
def confusion_matrix_heatmap(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

# Split

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

#dataset split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


# Optimization

In [8]:
Chi2 = []


In [9]:
from Metaheuristicas.Cuckoo import cuckoo_search
n = 50
dim = 84

for i in range(100,550,50):
    nests, fitness_scores, best_nest, best_CKMIfitness = cuckoo_search(n, dim, i, X_train, y_train, fitness_function=chi2_eval)
    CKMIFeatures = X.columns[best_nest.astype(bool)].tolist()

    Chi2.append([CKMIFeatures, best_CKMIfitness, i])

Mejor fitness al final de la iteración 1: 7.3746

Mejor fitness al final de la iteración 2: 8.6098
Mejor fitness al final de la iteración 3: 8.6098

Mejor fitness al final de la iteración 4: 8.6098
Mejor fitness al final de la iteración 5: 8.6098

Mejor fitness al final de la iteración 6: 8.6098

Mejor fitness al final de la iteración 7: 8.7726
Mejor fitness al final de la iteración 8: 8.7726

Mejor fitness al final de la iteración 9: 8.7726

Mejor fitness al final de la iteración 10: 8.7726
Mejor fitness al final de la iteración 11: 8.7726

Mejor fitness al final de la iteración 12: 8.7726
Mejor fitness al final de la iteración 13: 8.7726

Mejor fitness al final de la iteración 14: 8.7726
Mejor fitness al final de la iteración 15: 8.7739

Mejor fitness al final de la iteración 16: 8.7739
Mejor fitness al final de la iteración 17: 8.7739

Mejor fitness al final de la iteración 18: 8.7793

Mejor fitness al final de la iteración 19: 8.9967
Mejor fitness al final de la iteración 20: 8.996

In [21]:
GRX2Features, _, h = max(Chi2, key=lambda x: x[1])


## Red Neuronal


In [13]:
print("Features selected by X2: ", GRX2Features)


Features selected by X2:  ['f3_t_var', 'f7_t_time2peak', 'f8_t_rms', 'f14_f_peaks_pos_1', 'f15_f_90_percent_energy', 'f22_f_multiscaleEntropy', 'f23_f_peak_1020_value', 'f24_f_peak_1020_pos', 'f25_f_peak_2030_value', 'f26_f_peak_2030_pos', 'f28_f_peak2rms', 'f30_f_PeaksAboveRMSDensity_fun', 'f31_f_peaks_val_2', 'f32_f_peaks_pos_2', 'f34_f_peaks_pos_3', 'f36_w_f_maxval_D1', 'f37_w_f_maxval_D2', 'f38_w_f_maxval_D3', 'f43_w_f_maxpos_D2', 'f44_w_f_maxpos_D3', 'f51_w_f_mean_D3', 'f55_w_t_meanEnergyAD', 'f58_w_t_PEC_D2', 'f59_w_t_PEC_D3', 'f62_w_t_PEC_D6', 'f65_w_t_rms_D2', 'f70_w_t_peak2peak_A6', 'f71_w_t_peak2peak_D1', 'f72_w_t_peak2peak_D2', 'f75_w_t_peak2peak_D5', 'f76_w_t_peak2peak_D6', 'f79_w_t_peak2rms_D2', 'f80_w_t_peak2rms_D3', 'f82_w_t_peak2rms_D5']


In [14]:
print("Features selected by X2: ", len(GRX2Features))

Features selected by X2:  34


CHI2

In [19]:
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# 1. Split data into 75% train, 25% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# 2. Configure K-Fold Cross-Validation on the 75% training set
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics during cross-validation
accuracies, precisions, recalls, f1s, aucs = [], [], [], [], []

# 3. Perform 10-Fold Cross-Validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Define a new model in each fold
        # Define the model correctly
    input_dim = X_train.shape[1]  # Get the number of features
    
    model = Sequential([
        Input(shape=(input_dim,)),  # Instead of input_dim in Dense, use Input(shape=...)
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy'])

    # Train the model on the fold
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, verbose=0)

    # Evaluate on validation fold
    y_pred_prob = model.predict(X_val_fold)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Calculate metrics
    accuracies.append(accuracy_score(y_val_fold, y_pred))
    precisions.append(precision_score(y_val_fold, y_pred))
    recalls.append(recall_score(y_val_fold, y_pred))
    f1s.append(f1_score(y_val_fold, y_pred))
    aucs.append(roc_auc_score(y_val_fold, y_pred_prob))

# 4. Compute cross-validation average metrics
avg_acc = np.mean(accuracies)
avg_prec = np.mean(precisions)
avg_rec = np.mean(recalls)
avg_f1 = np.mean(f1s)
avg_auc = np.mean(aucs)

print("\nCross-Validation Results (on 75% training data):")
print(f'Accuracy: {avg_acc:.4f}, Precision: {avg_prec:.4f}, Recall: {avg_rec:.4f}, F1: {avg_f1:.4f}, AUC: {avg_auc:.4f}')

# 5. Train final model on all 75% training data
final_model = Sequential([
    Dense(64, activation='relu', input_dim=input_dim),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
final_model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy'])

final_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# 6. Evaluate final model on the 25% test set
y_test_prob = final_model.predict(X_test)
y_test_pred = (y_test_prob > 0.5).astype(int)

test_acc = accuracy_score(y_test, y_test_pred)
test_prec = precision_score(y_test, y_test_pred)
test_rec = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_prob)

print("\nFinal Evaluation on Test Set (25% Holdout Data):")
print(f'Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}, AUC: {test_auc:.4f}')


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Cross-Validation Results (on 75% training data):
Accuracy: 0.9616, Precision: 0.7790, Recall: 0.7239, F1: 0.7406, AUC: 0.9744


C:\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Final Evaluation on Test Set (25% Holdout Data):
Accuracy: 0.9617, Precision: 0.7692, Recall: 0.8000, F1: 0.7843, AUC: 0.9873
